In [9]:
import math
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense ,LSTM
import matplotlib.pyplot as plt
import math
from numpy.random import seed
seed(3)
import tensorflow as tf
tf.compat.v1.set_random_seed(2)

In [10]:
Epochs=np.linspace(25,35,11,dtype=int)
Nodes=np.linspace(25,35,11,dtype=int)
Batch=5
no=9

In [11]:
df1=pd.read_csv("/content/drive/MyDrive/States/NEW_MODEL/COVID-19 Cases(27-10-2021).csv")
p="/content/drive/MyDrive/States/NEW_MODEL/BIHAR"
df=df1[df1["Region"]=="Bihar"]

In [12]:
df

,S. No.,Date,Region,Confirmed Cases,Active Cases,Cured/Discharged,Death
2842,2843,01/01/1970,Bihar,725827,81.0,716088.0,9658.0
2843,2844,22/03/2020,Bihar,2,1.0,0.0,1.0
2844,2845,23/03/2020,Bihar,2,1.0,0.0,1.0
2845,2846,24/03/2020,Bihar,2,1.0,0.0,1.0
2846,2847,25/03/2020,Bihar,3,2.0,0.0,1.0
...,...,...,...,...,...,...,...
3407,3408,22/10/2021,Bihar,726042,30.0,716351.0,9661.0
3408,3409,23/10/2021,Bihar,726045,30.0,716354.0,9661.0
3409,3410,24/10/2021,Bihar,726054,36.0,716357.0,9661.0
3410,3411,25/10/2021,Bihar,726058,38.0,716359.0,9661.0


In [13]:
data=df.filter(["Active Cases"])
dataset=data[1:].values
training_data_len=math.ceil(len(dataset)*.9)

In [14]:
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(dataset)
l=70

train_data=scaled_data[0:training_data_len,:]
x_train=[]
y_train=[]

for i in range(l,len(train_data)):
  x_train.append(train_data[i-l:i,0])
  y_train.append(train_data[i,0])

x_train,y_train=np.array(x_train),np.array(y_train)
x_train=np.reshape(x_train,(len(x_train),len(x_train[0]),1))



In [15]:
for N in Nodes:
  for E in Epochs:
    if N!=0:
      model=Sequential()
      model.add(LSTM(N,return_sequences=False,input_shape=(len(x_train[0]),1)))
      # model.add(LSTM(10,return_sequences=False))
      model.add(Dense(12))
      model.add(Dense(1))
      model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy']) #mean_squared_error
      hist=model.fit(x_train,y_train,batch_size=Batch,epochs=E) #upto 50 epoch

      loss=hist.history["loss"]

      Testsss=p+"/Test/"+str(N)+"/"+str(E)+".xlsx"
      lossss=p+"/Train/"+str(N)+"/"+str(E)+".xlsx"
      Predsss=p+"/Pred/"+str(N)+"/"+str(E)+".xlsx"
      dict1={"Epochs":np.linspace(1,E,E),"Loss":loss}
      df_3=pd.DataFrame(dict1)
      xl=pd.ExcelWriter(lossss)
      df_3.to_excel(xl)
      xl.save()

      #test
      test_data=scaled_data[training_data_len-l:,:]
      x_test=[]
      y_test=dataset[training_data_len:,:]

      for i in range(l,len(test_data)):
        x_test.append(test_data[i-l:i,0])

      #convert data to numpy
      x_test=np.array(x_test)

      #reshape
      x_test=np.reshape(x_test,(len(x_test),len(x_test[0]),1))

      predictions=model.predict(x_test)
      Test_Accuracy=model.evaluate(x_test,predictions)
      predictions=scaler.inverse_transform(predictions)


      predictions1=np.reshape(predictions,(len(predictions)))

      dict1={"Days":np.linspace(len(x_train)+1,len(x_train)+len(x_test),len(x_test)),"Test Prediction":predictions1}
      df_3=pd.DataFrame(dict1)
      xl=pd.ExcelWriter(Testsss)
      df_3.to_excel(xl)
      xl.save()

      xx=np.reshape(x_test[-1],(1,l,1))
      prediction=model.predict(xx)
      pred=np.array([])
      for i in range(300):
        prediction=model.predict(xx)
        if prediction[0]<0:
          prediction[0]=xx[0][-1]
        pred=np.append(pred,prediction[0],axis=0)
        xx=np.delete(xx,0,axis=1)
        xx=np.append(xx[0],prediction,axis=0)
        xx=np.reshape(xx,(1,l,1))

      if len(pred)==0:
        continue

      pred=np.reshape(pred,(len(pred),1))
      pred=scaler.inverse_transform(pred)

      XX=np.linspace(len(dataset),len(dataset)+pred.size-1,pred.size)

      pred1=np.reshape(pred,(len(pred)))
      dict1={"Days":XX,"Third Wave":pred1}
      df_3=pd.DataFrame(dict1)
      xl=pd.ExcelWriter(Predsss)
      df_3.to_excel(xl)
      xl.save()
      print(N,E)


    

            


Epoch 1/25
89/89 [==============================] - 5s 27ms/step - loss: 0.0111 - accuracy: 0.0000e+00
Epoch 2/25
89/89 [==============================] - 2s 27ms/step - loss: 0.0019 - accuracy: 0.0023
Epoch 3/25
89/89 [==============================] - 2s 27ms/step - loss: 6.7397e-04 - accuracy: 0.0023
Epoch 4/25
89/89 [==============================] - 2s 27ms/step - loss: 4.7181e-04 - accuracy: 0.0023
Epoch 5/25
89/89 [==============================] - 2s 27ms/step - loss: 5.5490e-04 - accuracy: 0.0023
Epoch 6/25
89/89 [==============================] - 2s 28ms/step - loss: 4.3032e-04 - accuracy: 0.0023
Epoch 7/25
89/89 [==============================] - 2s 28ms/step - loss: 3.0969e-04 - accuracy: 0.0023
Epoch 8/25
89/89 [==============================] - 2s 27ms/step - loss: 3.6792e-04 - accuracy: 0.0023
Epoch 9/25
89/89 [==============================] - 2s 27ms/step - loss: 3.0190e-04 - accuracy: 0.0023
Epoch 10/25
89/89 [==============================] - 2s 28ms/step - loss: 2.8

FileNotFoundError: ignored